<a href="https://colab.research.google.com/github/anton0007/names-spanish/blob/master/modelo_clasificador_nombres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/anton0007/names-spanish


Cloning into 'names-spanish'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 45 (delta 22), reused 33 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 825.05 KiB | 8.17 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [3]:
# Cargamos los archivos csv con numpy para tenerlos listos
import numpy as np
import nltk, random
import unicodedata
tag_hombre = np.genfromtxt('/content/names-spanish/male_names.csv', skip_header=1, delimiter=',', dtype=('U20','i8','f8'))
tag_mujer = np.genfromtxt('/content/names-spanish/female_names.csv', skip_header=1, delimiter=',', dtype=('U20','i8','f8'))


In [4]:
# Verificamos si esta llamando a los nombres
print(tag_hombre[0])
print (tag_mujer[0])

('JOSE MARIA', 4785885, 65.)
('MARIA RENE', 74852, 19.)


In [5]:
#Juntamos las listas de hombres y mujeres
f_set = [(name[0], 'Masculino') for name in tag_hombre] + [(name [0], 'Femenino') for name in tag_mujer]

In [30]:
# Entrenamos teniendo en cuenta el nombre
def atributos2(nombre):
#nombre = nombre.lower().strip()  # Convertir a minúsculas y eliminar espacios al principio y al final
    atrib = {}
    # Últimas letras
    atrib["ultima_letra"] = nombre[-1].lower() if len(nombre) > 0 else ''
    atrib["ultimas_dos_letras"] = nombre[-2:].lower() if len(nombre) > 1 else ''
    atrib["ultimas_tres_letras"] = nombre[-3:].lower() if len(nombre) > 2 else ''
    atrib["ultimas_cuatro_letras"] = nombre[-4:].lower() if len(nombre) > 3 else ''
    atrib["ultimas_cinco_letras"] = nombre[-5:].lower() if len(nombre) > 4 else ''

    # Características generales
    atrib["cantidad_vocales"] = sum(1 for char in nombre if char in 'aeiou')
    atrib["cantidad_consonantes"] = sum(1 for char in nombre if char.isalpha() and char not in 'aeiou')
    atrib["longitud"] = len(nombre)
    atrib["primera_letra"] = nombre[0].lower() if len(nombre) > 0 else ''
    atrib["segunda_letra"] = nombre[1].lower() if len(nombre) > 1 else ''
    atrib["cantidad_digitos"] = sum(1 for char in nombre if char.isdigit())

    # Manejo de espacios y nombres compuestos
    componentes = nombre.split()
    num_componentes = len(componentes)
    atrib["num_componentes"] = num_componentes

    if num_componentes > 0:
        atrib["primer_componente"] = componentes[0]
    else:
        atrib["primer_componente"] = ''

    if num_componentes > 1:
        atrib["segundo_componente"] = componentes[1]
    else:
        atrib["segundo_componente"] = ''

    # Ejemplo de cómo manejar sufijos comunes (opcional, requiere lista de sufijos comunes)
    sufijos_comunes = ["es", "ía", "al", "an", "ar", "er", "or","a", "ina", "ela", "isa","ia", "ana", "ora", "ena", "ona", "ita","e","o", "el", "io", "ón", "or", "an", "ín", "és", "ato", "ino"]
    atrib["tiene_sufijo_comun"] = any(nombre.endswith(sufijo) for sufijo in sufijos_comunes)
    return atrib

f_varios_atributo = [(atributos2(n), g) for (n, g) in f_set]


In [31]:
from sklearn.model_selection import train_test_split
#Usamos el 80% de los datos para train y 20% para test
f_varios_atributo_train, f_varios_atributo_test = train_test_split(f_varios_atributo, test_size=0.20, random_state=45)

In [32]:
# Entrenamos el modelo classifier_2
classifier_2 = nltk.NaiveBayesClassifier.train(f_varios_atributo_train)

In [45]:
# Sacamos accuracy
print(classifier_2.classify(atributos2('Maria René')))
print("Probabilidad de:")
print(nltk.classify.accuracy(classifier_2, f_varios_atributo_test))

Femenino
Probabilidad de:
0.9254230418482116


In [49]:
import pickle

# Guardar tu modelo entrenado
with open('modelo_clasificador_nombres.pkl', 'wb') as f:
    pickle.dump(classifier_2, f)